In [1]:
import tensorflow as tf
import numpy as np


def round(x):
    return "%.2f" % np.round(x.numpy(), 2)


def SSIM(a, b):
    return round(tf.image.ssim(a, b, 255))


def PSNR(a, b):
    return round(tf.image.psnr(a, b, 255))

In [2]:
# Get validation images

from dlsr import *
from dlsr.data import DIV2K

# from PIL import Image

image_size = 400
loader = DIV2K(type="valid")
ds = loader.dataset(
    batch_size=1, random_transform=False, crop_images=True, image_size=image_size
)

hr = ds.map(lambda a, b: b).take(7).cache()
lr = ds.map(lambda a, b: a).take(7).cache()

upscaled = {}

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2023-02-09 16:37:10.868007: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-09 16:37:10.868032: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# # Upscale with my SRResNet

model = tf.keras.models.load_model(
    "./results/saved-models/srres2x.h5",
)

upscaled["srres2x"] = upscale(model, lr)

# Upscale with my SRGAN

# load json and create model
json_file = open('./results/saved-models/srgan2x/architecture.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = tf.keras.models.model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("./results/saved-models/srgan2x/generator.h5")

upscaled["srgan2x"] = upscale(model, lr)

2023-02-09 16:37:12.167893: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-09 16:37:12.395018: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/7 [===>..........................] - ETA: 5s

2023-02-09 16:37:12.873799: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2023-02-09 16:37:12.901070: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


7/7 [==============================] - 2s 203ms/step


2023-02-09 16:37:15.421443: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


7/7 [==============================] - 2s 194ms/step


In [4]:
# Upscale with Interploation-Based methods

lr_transformed = [np.uint8(x)[0] for x in lr]

new_size = [image_size, image_size]

algs = [
    tf.image.ResizeMethod.NEAREST_NEIGHBOR,
    tf.image.ResizeMethod.BILINEAR,
    tf.image.ResizeMethod.BICUBIC,
    tf.image.ResizeMethod.LANCZOS3,
]

for alg in algs:
    upscaled[alg] = tf.image.resize(
        lr_transformed,
        new_size,
        method=alg,
    )

In [5]:
# save LR images
for i, x in enumerate(lr):
    tf.keras.utils.save_img(path=f"results/comparison2x/lr/image{i}.png", x=x[0], file_format="png")

# save HR images
for i, x in enumerate(hr):
    tf.keras.utils.save_img(path=f"results/comparison2x/hr/image{i}.png", x=x[0], file_format="png")

# save upscaled images
for key in upscaled.keys():
    for i, x in enumerate(upscaled[key]):
        tf.keras.utils.save_img(
            path=f"results/comparison2x/{key}/image{i}.png", x=x, file_format="png"
        )

# calculate SSIM/PSNR
for i, x in enumerate(hr):
    real = np.uint8(x)[0]
    print("-------------------------------------")
    for j, key in enumerate(upscaled.keys()):
        fake = np.uint8(upscaled[key][i])
        print(
            "| {:<4} | {:<10} | {:<4} = {:<6} |".format(
                f"img{i}" if j == 0 else "", f"{key}", "SSIM", SSIM(real, fake)
            )
        )
        print("| {:<4} | {:<10} | {:<4} = {:<6} |".format("", "", "PSNR", PSNR(real, fake)))

2023-02-09 16:37:18.714048: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2023-02-09 16:37:18.719004: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


-------------------------------------
| img0 | srres2x    | SSIM = 0.99   |
|      |            | PSNR = 44.88  |
|      | srgan2x    | SSIM = 0.98   |
|      |            | PSNR = 42.13  |
|      | nearest    | SSIM = 0.95   |
|      |            | PSNR = 36.26  |
|      | bilinear   | SSIM = 0.98   |
|      |            | PSNR = 40.50  |
|      | bicubic    | SSIM = 0.99   |
|      |            | PSNR = 42.75  |
|      | lanczos3   | SSIM = 0.99   |
|      |            | PSNR = 43.47  |
-------------------------------------
| img1 | srres2x    | SSIM = 0.97   |
|      |            | PSNR = 42.42  |
|      | srgan2x    | SSIM = 0.93   |
|      |            | PSNR = 38.78  |
|      | nearest    | SSIM = 0.94   |
|      |            | PSNR = 37.43  |
|      | bilinear   | SSIM = 0.95   |
|      |            | PSNR = 38.74  |
|      | bicubic    | SSIM = 0.96   |
|      |            | PSNR = 39.01  |
|      | lanczos3   | SSIM = 0.96   |
|      |            | PSNR = 37.32  |
------------